In [59]:
# Librerias
import pandas as pd
import numpy as np
import re

In [60]:
# Dataset
df = pd.read_csv('hombres.csv')
#df = pd.read_csv('mujeres.csv')

#### Variables globales

In [61]:
# Rango y numero de elementos
rango = df.max() - df.min()
n = df.shape[0]

nombre_columna = df.columns[0]

# Numero de clases
if n < 30:
    numero_clases = int(np.ceil(np.sqrt(n)))
else:
    numero_clases = int(np.ceil(1 + (3.322 * np.log10(n))))

# Amplitud
amplitud = rango / numero_clases

#### Funciones por hacer

In [62]:
# Funcion para obtener la media
def media_agrupada(tabla_frecuencias, n):
    tabla_frecuencias['x_i * f_i'] = tabla_frecuencias['Marca de clase'] * tabla_frecuencias['Frecuencia absoluta']
    sum_xifi = float(tabla_frecuencias['x_i * f_i'].sum())
    media_agrupada = sum_xifi / n
    return media_agrupada

In [63]:
# Funcion para obtener la clase a la que pertenece la moda
def moda_clase(tabla_frecuencias):
    clase = 0
    frecuencia_mayor = tabla_frecuencias['Frecuencia absoluta'].max()

    for i in tabla_frecuencias['Frecuencia absoluta']:
            clase += 1
            if i == frecuencia_mayor:
                return clase

# Funcion para obtener la moda
def moda_agrupada(moda_clase, amplitud, tabla_frecuencias, intervalos_float):
    limite_inferior = min(intervalos_float[moda_clase - 1]) 
    delta_1 = tabla_frecuencias['Frecuencia absoluta'].iloc[moda_clase - 1] - tabla_frecuencias['Frecuencia absoluta'].iloc[moda_clase - 2]
    delta_2 = tabla_frecuencias['Frecuencia absoluta'].iloc[moda_clase - 1] - tabla_frecuencias['Frecuencia absoluta'].iloc[moda_clase]
    moda_agrupada = limite_inferior + ((delta_1) / (delta_1 + delta_2)) * amplitud
    return moda_agrupada

In [64]:
# Funcion para la posicion de la clase en la que se encuentra la mediana
def mediana_clase(n, tabla_frecuencias):
    posicion = (n + 1) / 2
    clase = 0

    posicion_clase = next((clase + 1 for clase, frecuencia_acumulada in enumerate(tabla_frecuencias['Frecuencia acumulada']) if frecuencia_acumulada>=posicion), None)
    
    return posicion_clase

# Funcion para obtener la mediana
def mediana_agrupada(mediana_clase, amplitud, tabla_frecuencias, intervalos_float):
    limite_inferior = min(intervalos_float[mediana_clase - 1]) 
    frec_acum_anterior = tabla_frecuencias['Frecuencia acumulada'].iloc[mediana_clase - 2]
    frec_absoluta = tabla_frecuencias['Frecuencia absoluta'].iloc[mediana_clase -1]
    mediana_agrupada = limite_inferior + (((n / 2) - frec_acum_anterior) / frec_absoluta ) * amplitud

    return mediana_agrupada

In [65]:
# Funcion para obtener la varianza
def varianza_agrupada(tabla_frecuencias, n, media_agrupada):
    tabla_frecuencias['(x_i - \mu)^2'] = ((tabla_frecuencias['Marca de clase'] - media_agrupada) ** 2)
    tabla_frecuencias['(x_i - \mu)^2 * f_i'] = tabla_frecuencias['(x_i - \mu)^2'] * tabla_frecuencias['Frecuencia absoluta']
    sumatoria = tabla_frecuencias['(x_i - \mu)^2 * f_i'].sum()
    varianza_agrupada = (sumatoria) / (n - 1)
    return varianza_agrupada

In [66]:
# Funcion para obtener el rango
def rango_agrupado(tabla_frecuencias):
    rango = tabla_frecuencias['Marca de clase'].max() - tabla_frecuencias['Marca de clase'].min()
    return rango

#### Tabla de datos agrupados

In [67]:
nombre_columna = df.columns[0]
clases = df[nombre_columna].value_counts(bins = numero_clases, sort = False)
tabla_frecuencias = pd.DataFrame(clases)
tabla_frecuencias.rename(columns = {'count':'Frecuencia absoluta'}, inplace = True)
tabla_frecuencias
clases

HOMBRES 
(55.857, 61.871]     6
(61.871, 67.843]    10
(67.843, 73.814]    23
(73.814, 79.786]     8
(79.786, 85.757]     4
(85.757, 91.729]     5
(91.729, 97.7]       1
Name: count, dtype: int64

In [68]:
df[df.columns[0]].values
df.max()

HOMBRES     97.7
dtype: float64

In [69]:
array_classes = np.empty((numero_clases, 2), dtype=float)
array_classes[0][0] = df.min().iloc[0] 
array_classes[0][1] = df.min().iloc[0] + amplitud.iloc[0]
aux_col1 = df.min().iloc[0]
aux_col2 = df.min().iloc[0] + amplitud.iloc[0]

for i in range(1,numero_clases):
    aux_col1 += amplitud
    array_classes[i][0] = aux_col1.iloc[0]
        
for i in range(1,numero_clases):
    aux_col2 += amplitud
    array_classes[i][1] = aux_col2.iloc[0]

array_classes

array([[55.9       , 61.87142857],
       [61.87142857, 67.84285714],
       [67.84285714, 73.81428571],
       [73.81428571, 79.78571429],
       [79.78571429, 85.75714286],
       [85.75714286, 91.72857143],
       [91.72857143, 97.7       ]])

In [70]:
array_abs_freq = np.empty((numero_clases), dtype = int)

'''
def last_interval(array, interval):
	counts = 0
	for i in array:
		if np.float32(interval[0]) <= np.float32(i) <= np.float32(interval[1]):
			counts += 1
	return counts
'''
    
def count_range_in_list(array, interval):
	counts = 0

	for i in array:
		if np.float32(i) != np.float32(array[-1]):
			if np.float32(interval[0]) <= np.float32(i) < np.float32(interval[1]):
				counts += 1
		else:
			if np.float32(interval[0]) <= np.float32(i) <= np.float32(interval[1]):
				counts += 1

	return counts

for i in range(numero_clases):
	array_abs_freq[i] = count_range_in_list(df[df.columns[0]].values, array_classes[i])

#array_abs_freq[-1] = last_interval(df[df.columns[0]].values, array_classes[-1])

'''
comparison = array_abs_freq == np.array(clases.values)
equal_arrays = comparison.all()
comparison
'''

print(array_abs_freq)

[ 6 10 23  8  4  5  1]


In [71]:
# Por terminar, marca de clase, f_i * x_i, ..., optimizar codigo
nombre_columna = df.columns[0]
clases = df[nombre_columna].value_counts(bins = numero_clases, sort = False)
tabla_frecuencias = pd.DataFrame(clases)
tabla_frecuencias.rename(columns = {'count':'Frecuencia absoluta'}, inplace = True)
frecuencia_acumulada = []
acumulador = 0

# Columna: frecuencia acumulada
for i in tabla_frecuencias.index:
    acumulador = acumulador + tabla_frecuencias['Frecuencia absoluta'][i]
    frecuencia_acumulada.append(acumulador)

# Columnas: f. acumulada y f. relativa
tabla_frecuencias['Frecuencia acumulada'] = frecuencia_acumulada
tabla_frecuencias['Frecuencia relativa absoluta'] = tabla_frecuencias['Frecuencia absoluta'] / n

# Arreglo de intervalos
# "[0-9]+.[0-9]+, [0-9]+.[0-9]+"gm
intervalos_str = []
intervalos_float = []
marca_clase = []

clases_lista = clases.index

for i in clases_lista:
    matched = re.search("[0-9]+.[0-9]+, [0-9]+.[0-9]+", str(i))
    intervalos_str.append(matched.group())
    
for i in intervalos_str:
    intervalos_float.append(i.split(', '))

for i in intervalos_float:
    for j in i:
        j = float(j)

intervalos_float = np.array(intervalos_float, dtype = float)

for i in intervalos_float:
    marca_clase.append(np.sum(i) / 2)

# Columna: frecuencia acumulada relativa
tabla_frecuencias['Frecuencia relativa acumulada'] = tabla_frecuencias['Frecuencia acumulada'] / n

# Columna: marca de clase
tabla_frecuencias['Marca de clase'] = marca_clase

# Columna: porcentaje
tabla_frecuencias['Porcentaje'] = tabla_frecuencias['Frecuencia relativa absoluta'] * 100

tabla_frecuencias

,Frecuencia absoluta,Frecuencia acumulada,Frecuencia relativa absoluta,Frecuencia relativa acumulada,Marca de clase,Porcentaje
HOMBRES,,,,,,
"(55.857, 61.871]",6,6,0.105263,0.105263,58.8640,10.526316
"(61.871, 67.843]",10,16,0.175439,0.280702,64.8570,17.543860
"(67.843, 73.814]",23,39,0.403509,0.684211,70.8285,40.350877
"(73.814, 79.786]",8,47,0.140351,0.824561,76.8000,14.035088
"(79.786, 85.757]",4,51,0.070175,0.894737,82.7715,7.017544
"(85.757, 91.729]",5,56,0.087719,0.982456,88.7430,8.771930
"(91.729, 97.7]",1,57,0.017544,1.000000,94.7145,1.754386


In [72]:
tabla_frecuencias.to_csv('tabla_frecuencias_' + df.columns[0] + '.csv')

#### Funciones test

In [73]:
rango_agrupado = rango_agrupado(tabla_frecuencias)
moda_clase = moda_clase(tabla_frecuencias)
moda_agrupada = moda_agrupada(moda_clase, amplitud, tabla_frecuencias, intervalos_float)
mediana_clase = mediana_clase(n, tabla_frecuencias)
mediana_agrupada = mediana_agrupada(mediana_clase, amplitud, tabla_frecuencias, intervalos_float)
media_agrupada = media_agrupada(tabla_frecuencias, n)
varianza_agrupada = varianza_agrupada(tabla_frecuencias, n, media_agrupada)
medidas_arreglo = [media_agrupada, float(mediana_agrupada), float(moda_agrupada), varianza_agrupada, rango_agrupado, float(df.max() - df.min())]
#python_arreglo = [df.columns[0].mean(), df.columns[0].median(), df.columns[0].mode().iloc[0], df.columns[0].var(ddof = 1), 0]

medidas_funciones = pd.DataFrame(medidas_arreglo, columns = ['Medidas de tendencia central: ' + df.columns[0].lower()], index = ['Media', 'Mediana', 'Moda', 'Varianza', 'Rango', 'Rango real'])
#medidas_python = pd.DataFrame(python_arreglo, columns = ['Valor real'])

medidas_funciones

/tmp/ipykernel_1909/349951850.py:8: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  medidas_arreglo = [media_agrupada, float(mediana_agrupada), float(moda_agrupada), varianza_agrupada, rango_agrupado, float(df.max() - df.min())]


,Medidas de tendencia central: hombres
Media,72.188158
Mediana,71.088342
Moda,70.615449
Varianza,73.948367
Rango,35.850500
Rango real,41.800000


In [74]:
medidas_funciones.to_csv('tabla_resultados_' + df.columns[0] + '.csv')

In [75]:
df.var(ddof = 1)

HOMBRES     71.754292
dtype: float64

In [76]:
df.describe()

,HOMBRES
count,57.000000
mean,71.943860
std,8.470791
min,55.900000
25%,65.900000
50%,70.500000
75%,77.300000
max,97.700000
